In [2]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('/kaggle/input/dataset-files/article_with_summary-tfidf.csv')
df2 = pd.read_csv('/kaggle/input/dataset-files/CNN_3000 translated_Hindi_IndicTrans_main.csv')
# Display the DataFrame
print(df)
df2

                                          article_hindi  \
0     लंदन, इंग्लैंड (रॉयटर्स)-हैरी पॉटर स्टार डैनिय...   
1     संपादक का नोटः हमारी बिहाइंड द सीन्स श्रृंखला ...   
2     मिनियापोलिस, मिनेसोटा (सी. एन. एन.)-एक ड्राइवर...   
3     वाशिंगटन (सी. एन. एन.)-डॉक्टरों ने शनिवार को र...   
4     (सी. एन. एन.)-नेशनल फुटबॉल लीग ने बिना वेतन के...   
...                                                 ...   
2995  (सी. एन. एन.)-एमी विजेता पैट्रिक मैकगुहान, अभि...   
2996  (सी. एन.)-- देखो! अपने प्रियजनों को बंद करो। य...   
2997  लॉस एंजेलिस, कैलिफोर्निया (सी. एन. एन.)-अमेरिक...   
2998  (सी. एन.)-जल्द या बाद में, सी. एन. एन. के प्रव...   
2999  (सी. एन. एन.)-पिछले सफल विमानों की खाई से सीखन...   

                                         tf_idf_summary  
0     लंदन, इंग्लैंड (रॉयटर्स)-हैरी पॉटर स्टार डैनिय...  
1     संपादक का नोटः हमारी बिहाइंड द सीन्स श्रृंखला ...  
2     मिनियापोलिस, मिनेसोटा (सी| वे वास्तव में जीवित...  
3     वाशिंगटन (सी| )-डॉक्टरों ने शनिवार को राष्ट्रप...  
4

,id,article_hindi,highlights_hindi
0,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4,"लंदन, इंग्लैंड (रॉयटर्स)-हैरी पॉटर स्टार डैनिय...",हैरी पॉटर स्टार डेनियल रैडक्लिफ को सोमवार को 1...
1,ee8871b15c50d0db17b0179a6d2beab35065f1e9,संपादक का नोटः हमारी बिहाइंड द सीन्स श्रृंखला ...,"मियामी में मानसिक रूप से बीमार कैदियों को ""भुल..."
2,06352019a19ae31e527f37f7571c6dd7f0c5da37,"मिनियापोलिस, मिनेसोटा (सी. एन. एन.)-एक ड्राइवर...","नयाः ""मुझे लगा कि मैं मरने वाला हूँ"", ड्राइवर ..."
3,24521a2abb2e1f5e34e6824e0f9e56904a2b0e88,वाशिंगटन (सी. एन. एन.)-डॉक्टरों ने शनिवार को र...,"प्रक्रिया के दौरान पाँच छोटे पॉलीप पाए गए; ""को..."
4,7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a,(सी. एन. एन.)-नेशनल फुटबॉल लीग ने बिना वेतन के...,"NEW: एन. एफ. एल. प्रमुख, अटलांटा फाल्कन्स के म..."
...,...,...,...
2995,9a0c293f566117ca98df29881cb3c8d602ce3221,"(सी. एन. एन.)-एमी विजेता पैट्रिक मैकगुहान, अभि...",अभिनेता पैट्रिक मैकगूहान का 80 वर्ष की आयु में...
2996,022174bd222ecb5e9030dcb18b1ccf19f1f95ab8,(सी. एन.)-- देखो! अपने प्रियजनों को बंद करो। य...,"""मॉन्स्टर्स वर्सेज एलियन्स"" पृथ्वी पर जन्मे ""र..."
2997,caaa16986766cffbdd942c9bde92b3e5a75e3d25,"लॉस एंजेलिस, कैलिफोर्निया (सी. एन. एन.)-अमेरिक...",दक्षिणी कैलिफोर्निया गुरुवार शाम को हल्के भूकं...
2998,054120d8b1df99fd7a0a08595b6e9a30a8902289,"(सी. एन.)-जल्द या बाद में, सी. एन. एन. के प्रव...",कांग्रेस 12 जून तक देश भर में डिजिटल टीवी पर स...


In [3]:
from sentence_transformers import SentenceTransformer, util

# Load the pre-trained Sentence Transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Initialize variables for computing the average similarity score
final_val = 0
count = 0

# Compute embedding for both lists
ground = df2['highlights_hindi']
SystemSummary1 = df['tf_idf_summary']

for i in range(len(ground)):
    # Check for NaN values in both ground and generated summaries
    if pd.isna(ground[i]) or pd.isna(SystemSummary1[i]):
        continue  # Skip this iteration if any summary is NaN

    # Compute embeddings for both the ground summary and generated summary
    embedding_1 = model.encode(ground[i], convert_to_tensor=True)
    embedding_2 = model.encode(SystemSummary1[i], convert_to_tensor=True)

    # Compute cosine similarity between the embeddings
    val = util.pytorch_cos_sim(embedding_1, embedding_2)

    # Accumulate the similarity score
    final_val += val.item()
    count += 1

# Compute the average similarity score
if count > 0:
    final_val /= count

# Print the average similarity score
print("Average similarity score:", final_val)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 

The secret `HF_TOKEN` does not exist in your Colab secrets.

To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.

You will be able to reuse this secret in all of your notebooks.

Please note that authentication is recommended but still optional to access public models or datasets.

  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Average similarity score: 0.7313403066471219


In [5]:
!pip install rouge

In [6]:
from rouge import Rouge
rouge = Rouge()
score = rouge.get_scores(df['tf_idf_summary'], df2['highlights_hindi'], avg=True)
tFIDF = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
tFIDF

,rouge-1,rouge-2,rouge-l
recall,0.406837,0.154813,0.281068
precision,0.226605,0.071947,0.157232
f-measure,0.279576,0.094720,0.192962
